# NASA Ocean Data Coherence Checker
## SWOT x MODIS Integration for Shark Tracking

Este notebook testa a coerência entre dados SWOT e MODIS para rastreamento de tubarões.


In [ ]:
# === 1. Imports ===
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from datetime import datetime
from pandas import to_datetime
import os

# Criar diretório de resultados
os.makedirs("results", exist_ok=True)

print("✅ Bibliotecas importadas com sucesso!")


: 

In [ ]:
# === 2. Load datasets ===
# Ajustando caminhos para os arquivos disponíveis
swot_path = "SWOT_L2_LR_SSH_Expert_008_497_20240101T000705_20240101T005833_PIC0_01.nc"
modis_path = "AQUA_MODIS.20240101.L3b.DAY.AT203.nc" 

print("Loading SWOT:", swot_path)
try:
    ds_swot = xr.open_dataset(swot_path)
    print("✅ SWOT dataset carregado com sucesso!")
    print(f"   Dimensões: {ds_swot.dims}")
    print(f"   Variáveis: {len(ds_swot.data_vars)}")
except Exception as e:
    print(f"❌ Erro ao carregar SWOT: {e}")
    ds_swot = None

print("\nLoading MODIS:", modis_path)
try:
    ds_modis = xr.open_dataset(modis_path)
    print("✅ MODIS dataset carregado com sucesso!")
    print(f"   Dimensões: {ds_modis.dims}")
    print(f"   Variáveis: {len(ds_modis.data_vars)}")
except Exception as e:
    print(f"❌ Erro ao carregar MODIS: {e}")
    print("   Nota: Este arquivo não existe ainda. Vamos simular dados MODIS para teste.")
    ds_modis = None


In [ ]:
# === 3. Helper functions ===
def get_lat_lon_names(ds):
    """Encontra nomes das coordenadas de latitude e longitude"""
    lats = [c for c in ds.coords if 'lat' in c.lower()]
    lons = [c for c in ds.coords if 'lon' in c.lower()]
    return lats[0] if lats else None, lons[0] if lons else None

def get_bbox(ds):
    """Calcula bounding box do dataset"""
    lat_name, lon_name = get_lat_lon_names(ds)
    if lat_name is None or lon_name is None:
        return None
    lats = ds[lat_name].values
    lons = ds[lon_name].values
    return np.nanmin(lats), np.nanmax(lats), np.nanmin(lons), np.nanmax(lons)

def get_time_range(ds):
    """Encontra range temporal do dataset"""
    # Procura primeiro nas coordenadas
    tname = [c for c in ds.coords if 'time' in c.lower()]
    if not tname:
        # Se não encontrar nas coordenadas, procura nas variáveis de dados
        tname = [v for v in ds.data_vars if 'time' in v.lower()]
    
    if not tname:
        return None
    
    t = ds[tname[0]].values
    return np.min(t), np.max(t)

def bbox_intersection(b1, b2):
    """Calcula interseção entre dois bounding boxes"""
    if b1 is None or b2 is None:
        return None
    
    lat_min = max(b1[0], b2[0])
    lat_max = min(b1[1], b2[1])
    lon_min = max(b1[2], b2[2])
    lon_max = min(b1[3], b2[3])
    
    if lat_min >= lat_max or lon_min >= lon_max:
        return None
    return (lat_min, lat_max, lon_min, lon_max)

print("✅ Funções auxiliares definidas!")


In [ ]:
# === 4. Análise dos dados SWOT ===
if ds_swot is not None:
    print("=== ANÁLISE SWOT ===")
    swot_bbox = get_bbox(ds_swot)
    swot_time = get_time_range(ds_swot)
    
    print("SWOT BBOX:", swot_bbox)
    print("SWOT Time range:", swot_time)
    print("SWOT Coordinates:", list(ds_swot.coords.keys()))
    print("SWOT Data variables:", list(ds_swot.data_vars.keys())[:5], "...")  # Primeiras 5 variáveis
    
    # Mostrar informações detalhadas
    print(f"\nDimensões SWOT: {ds_swot.dims}")
    print(f"Tamanho do dataset: {ds_swot.nbytes / 1024**2:.1f} MB")
else:
    print("❌ Dataset SWOT não disponível")
    swot_bbox = None
    swot_time = None


In [ ]:
# === 5. Simulação de dados MODIS para teste ===
if ds_modis is None:
    print("=== SIMULANDO DADOS MODIS PARA TESTE ===")
    
    # Criar dados MODIS simulados baseados no SWOT
    if ds_swot is not None:
        # Usar as mesmas coordenadas do SWOT para simular overlap
        lat_swot = ds_swot.latitude.values
        lon_swot = ds_swot.longitude.values
        
        # Criar dataset MODIS simulado
        ds_modis = xr.Dataset({
            'chlor_a': (['lat', 'lon'], np.random.lognormal(0, 0.5, lat_swot.shape))
        }, coords={
            'lat': (['lat'], lat_swot[:, 0]),  # Usar primeira coluna de latitude
            'lon': (['lon'], lon_swot[0, :]),  # Usar primeira linha de longitude
            'time': [np.datetime64('2024-01-01T12:00:00')]
        })
        
        print("✅ Dataset MODIS simulado criado!")
        print(f"   Dimensões: {ds_modis.dims}")
        print(f"   Variáveis: {list(ds_modis.data_vars.keys())}")
    else:
        print("❌ Não é possível simular MODIS sem dados SWOT")

# Análise dos dados MODIS (reais ou simulados)
if ds_modis is not None:
    print("\n=== ANÁLISE MODIS ===")
    modis_bbox = get_bbox(ds_modis)
    modis_time = get_time_range(ds_modis)
    
    print("MODIS BBOX:", modis_bbox)
    print("MODIS Time range:", modis_time)
    print("MODIS Coordinates:", list(ds_modis.coords.keys()))
    print("MODIS Data variables:", list(ds_modis.data_vars.keys()))
else:
    print("❌ Dataset MODIS não disponível")
    modis_bbox = None
    modis_time = None


In [ ]:
# === 6. Verificação de interseção espacial ===
print("=== VERIFICAÇÃO DE INTERSEÇÃO ===")

if swot_bbox is not None and modis_bbox is not None:
    intersection = bbox_intersection(swot_bbox, modis_bbox)
    
    if intersection:
        print("✅ Interseção espacial encontrada:", intersection)
        print(f"   Área de overlap: {intersection[1]-intersection[0]:.2f}° lat × {intersection[3]-intersection[2]:.2f}° lon")
    else:
        print("❌ Nenhuma interseção espacial encontrada")
        print("   Os datasets não se sobrepõem geograficamente")
else:
    print("❌ Não é possível verificar interseção - dados insuficientes")
    intersection = None


In [ ]:
# === 7. Visualização dos bounding boxes ===
if swot_bbox is not None and modis_bbox is not None:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Plot SWOT bounding box
    ax.add_patch(patches.Rectangle((swot_bbox[2], swot_bbox[0]),
                                   swot_bbox[3]-swot_bbox[2],
                                   swot_bbox[1]-swot_bbox[0],
                                   linewidth=2, edgecolor='blue', facecolor='lightblue', 
                                   alpha=0.3, label='SWOT'))
    
    # Plot MODIS bounding box
    ax.add_patch(patches.Rectangle((modis_bbox[2], modis_bbox[0]),
                                   modis_bbox[3]-modis_bbox[2],
                                   modis_bbox[1]-modis_bbox[0],
                                   linewidth=2, edgecolor='green', facecolor='lightgreen', 
                                   alpha=0.3, label='MODIS'))
    
    # Plot intersection if exists
    if intersection:
        ax.add_patch(patches.Rectangle((intersection[2], intersection[0]),
                                       intersection[3]-intersection[2],
                                       intersection[1]-intersection[0],
                                       linewidth=2, edgecolor='red', facecolor='red', 
                                       alpha=0.5, label='Interseção'))
    
    ax.set_xlabel("Longitude (°)")
    ax.set_ylabel("Latitude (°)")
    ax.legend()
    ax.set_title("Bounding Boxes: SWOT vs MODIS")
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig("results/bounding_boxes.png", dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✅ Gráfico salvo em results/bounding_boxes.png")
else:
    print("❌ Não é possível criar visualização - dados insuficientes")


In [ ]:
# === 8. Análise de correlação (se houver interseção) ===
if intersection and ds_swot is not None and ds_modis is not None:
    print("=== ANÁLISE DE CORRELAÇÃO ===")
    
    # Definir área de amostragem
    lat_min, lat_max, lon_min, lon_max = intersection
    n_points = 100  # Reduzido para teste mais rápido
    
    # Gerar pontos aleatórios na área de interseção
    np.random.seed(42)
    lats = np.random.uniform(lat_min, lat_max, n_points)
    lons = np.random.uniform(lon_min, lon_max, n_points)
    
    # Escolher variáveis para análise
    swot_var = 'ssha_karin' if 'ssha_karin' in ds_swot.data_vars else list(ds_swot.data_vars)[0]
    modis_var = 'chlor_a' if 'chlor_a' in ds_modis.data_vars else list(ds_modis.data_vars)[0]
    
    print(f"Variável SWOT: {swot_var}")
    print(f"Variável MODIS: {modis_var}")
    
    # Amostrar valores
    vals = []
    for la, lo in zip(lats, lons):
        try:
            # Interpolar valores do SWOT
            ssha = ds_swot[swot_var].interp(latitude=la, longitude=lo, method='nearest')
            # Interpolar valores do MODIS
            chl = ds_modis[modis_var].interp(lat=la, lon=lo, method='nearest')
            vals.append((float(la), float(lo), float(ssha), float(chl)))
        except Exception as e:
            vals.append((la, lo, np.nan, np.nan))
    
    # Criar DataFrame
    df = pd.DataFrame(vals, columns=["lat", "lon", "ssha", "chlorophyll"])
    df_clean = df.dropna()
    
    print(f"Pontos válidos: {len(df_clean)}/{len(df)}")
    
    if len(df_clean) > 10:  # Mínimo de pontos para correlação
        # Calcular correlação
        corr = df_clean["ssha"].corr(df_clean["chlorophyll"])
        print(f"Correlação Pearson (SSHA vs Chl): {corr:.3f}")
        
        # Criar scatter plot
        plt.figure(figsize=(8, 6))
        plt.scatter(df_clean["ssha"], df_clean["chlorophyll"], s=20, c="steelblue", alpha=0.6)
        plt.title(f"SSHA vs Chlorophyll\nCorrelação = {corr:.3f}")
        plt.xlabel("SSHA (m)")
        plt.ylabel("Chlorophyll (mg/m³)")
        plt.grid(True, alpha=0.3)
        
        # Adicionar linha de tendência
        z = np.polyfit(df_clean["ssha"], df_clean["chlorophyll"], 1)
        p = np.poly1d(z)
        plt.plot(df_clean["ssha"], p(df_clean["ssha"]), "r--", alpha=0.8)
        
        plt.tight_layout()
        plt.savefig("results/scatter_ssha_chl.png", dpi=150, bbox_inches='tight')
        plt.show()
        
        print("✅ Gráfico de correlação salvo em results/scatter_ssha_chl.png")
    else:
        print("❌ Pontos insuficientes para análise de correlação")
        corr = np.nan
else:
    print("❌ Não é possível fazer análise de correlação - sem interseção ou dados insuficientes")
    corr = np.nan


In [ ]:
# === 9. Relatório final ===
print("=== RELATÓRIO FINAL ===")

# Criar relatório
report = {
    "timestamp": [datetime.now().strftime("%Y-%m-%d %H:%M:%S")],
    "swot_file": [swot_path if ds_swot is not None else "N/A"],
    "modis_file": [modis_path if ds_modis is not None else "N/A"],
    "swot_loaded": [ds_swot is not None],
    "modis_loaded": [ds_modis is not None],
    "swot_bbox": [str(swot_bbox) if swot_bbox else "N/A"],
    "modis_bbox": [str(modis_bbox) if modis_bbox else "N/A"],
    "intersection_found": [intersection is not None],
    "intersection_bbox": [str(intersection) if intersection else "N/A"],
    "swot_time_range": [str(swot_time) if swot_time else "N/A"],
    "modis_time_range": [str(modis_time) if modis_time else "N/A"],
    "correlation_ssha_chl": [corr if not np.isnan(corr) else "N/A"],
    "valid_samples": [len(df_clean) if 'df_clean' in locals() else 0]
}

# Salvar relatório
df_report = pd.DataFrame(report)
df_report.to_csv("results/analysis_report.csv", index=False)

print("📊 RESUMO DOS RESULTADOS:")
print(f"   • SWOT carregado: {'✅' if ds_swot is not None else '❌'}")
print(f"   • MODIS carregado: {'✅' if ds_modis is not None else '❌'}")
print(f"   • Interseção espacial: {'✅' if intersection else '❌'}")
print(f"   • Correlação calculada: {'✅' if not np.isnan(corr) else '❌'}")
if not np.isnan(corr):
    print(f"   • Correlação SSHA-Chl: {corr:.3f}")

print(f"\n✅ Relatório salvo em results/analysis_report.csv")
print(f"✅ Gráficos salvos em results/")

# Mostrar próximos passos
print(f"\n📋 PRÓXIMOS PASSOS:")
if ds_modis is None:
    print("   1. Baixar dados MODIS reais da NASA")
    print("   2. Verificar se há overlap temporal e espacial")
if intersection is None:
    print("   3. Ajustar área de estudo para ter overlap")
if np.isnan(corr):
    print("   4. Verificar qualidade dos dados para correlação")
else:
    print("   5. Analisar significância estatística da correlação")
    print("   6. Expandir análise para mais variáveis")
